In [1]:
cd /content/drive/My\ Drive/Transformer/

/content/drive/My Drive/Transformer


# ライブラリ読み込み

In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [3]:
import numpy as np
import os
import time
import MeCab

import preprocess_utils
import model
import weight_utils

import tensorflow.keras as keras
import tensorflow as tf
print(tf.__version__)

2.3.0


# 日英翻訳データ ダウンロード

In [ ]:
!wget http://www.manythings.org/anki/jpn-eng.zip
!unzip ./jpn-eng.zip

# データ読み込み

In [4]:
dataset = preprocess_utils.CreateData(
    corpus_path = './jpn.txt',
    do_shuffle=True,
    seed_value=100,
    split_percent=0.95 # 学習データの割合
)

train_source, train_target, test_source, test_target = dataset.split_data()

print('**** Amount of data ****')
print('train_source： ', len(train_source))
print('train_target： ', len(train_target))
print('test_source： ', len(test_source))
print('test_target： ', len(test_target))
print('\n')
print('**** Train data example ****')
print('Source Example： ', train_source[0])
print('Target Example： ', train_target[0])
print('\n')
print('**** Test data example ****')
print('Source Example： ', test_source[0])
print('Target Example： ', test_target[0])

**** Amount of data ****
train_source：  50914
train_target：  50914
test_source：  2680
test_target：  2680


**** Train data example ****
Source Example：  Tom loves spaghetti.
Target Example：  トムはスパゲッティが大好きです。


**** Test data example ****
Source Example：  Being left alone, the boy didn't know what to do.
Target Example：  ひとりぼっちにされて少年はどうしてよいかわからなかった。


# 前処理

In [5]:
BATCH_SIZE = 64 # バッチサイズ
MAX_LANGTH = 60 # シーケンスの長さ
USE_TPU = False # TPUを使うか
BUFFER_SIZE = 50000

In [6]:
train_dataset = preprocess_utils.PreprocessData(
    mecab = MeCab.Tagger("-Ochasen"),
    source_data = train_source,
    target_data = train_target,
    max_length = MAX_LANGTH,
    batch_size = BATCH_SIZE,
    test_flag = False,
    train_dataset = None,
)

train_dataset.preprocess_data()

In [7]:
if USE_TPU:
  tpu_grpc_url = "grpc://" + os.environ["COLAB_TPU_ADDR"]
  tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_grpc_url)
  tf.config.experimental_connect_to_cluster(tpu_cluster_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_cluster_resolver)    
  strategy = tf.distribute.experimental.TPUStrategy(tpu_cluster_resolver)

trainset = tf.data.Dataset.from_tensor_slices((train_dataset.source_vector, train_dataset.target_vector))
trainset = trainset.map(lambda source, target: (tf.cast(source, tf.int64), tf.cast(target, tf.int64))).shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

if USE_TPU:
  trainset = strategy.experimental_distribute_dataset(trainset)

# モデル定義

In [8]:
num_layers=4 # レイヤー数
d_model=64 # 中間層のユニット数
num_heads=4 # Multi Head Attentionのヘッド数
dff=2048 # Feed Forward Networkのユニット数
dropout_rate = 0.1 # ドロップアウト率

source_vocab_size = max(train_dataset.source_token.values()) + 1 # source文の語彙数
target_vocab_size = max(train_dataset.target_token.values()) + 1 # target文の語彙数

In [9]:
# 重み初期化
def initialize_weight(checkpoint_path, optimizer, transformer, max_length, batch_size, use_tpu=False):

  if os.path.exists(checkpoint_path+'.pkl'):
    if use_tpu:
      number_of_tpu_cores = tpu_cluster_resolver.num_accelerators()['TPU']
      initialize_source, initialize_target = [[1]*max_length]*number_of_tpu_cores, [[1]*max_length]*number_of_tpu_cores
      initialize_set = tf.data.Dataset.from_tensor_slices((initialize_source, initialize_target))
      initialize_set = initialize_set.map(lambda source, target: (tf.cast(source, tf.int64), tf.cast(target, tf.int64))
          ).shuffle(buffer_size=BUFFER_SIZE).batch(batch_size).prefetch(
              buffer_size=tf.data.experimental.AUTOTUNE
          )
      initialize_set = strategy.experimental_distribute_dataset(initialize_set)

      for inp, tar in initialize_set:
        distributed_train_on_batch(inp, tar)

    else:
      initialize_set = tf.ones([batch_size, max_length], tf.int64)
      train_step(initialize_set, initialize_set)
    
    try:
      weight_utils.load_weights_from_pickle(checkpoint_path, optimizer, transformer)
    except:
      print('Failed to load checkpoints.')

  else:
    print('No available checkpoints.')

# 学習実行

In [ ]:
# Transformer
transformer = model.Transformer(num_layers, d_model, num_heads, dff,
                          source_vocab_size, target_vocab_size, 
                          pe_input=source_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

# learning rate
learning_rate = model.CustomSchedule(d_model)

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

# loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

# Loss Function
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

# metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# checkpoint
checkpoint_path = "/content/drive/My Drive/Transformer/checkpoints/gpu/model"

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]
@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = model.create_masks(inp, tar_inp)
  
  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

# initialize weight
initialize_weight(checkpoint_path, optimizer, transformer, MAX_LANGTH, BATCH_SIZE, use_tpu=USE_TPU)

EPOCHS = 30
batch = 0

for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  for inp, tar in trainset:
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
    batch+=1
      
  if (epoch + 1) % 5 == 0:
    print('Saving checkpoint for epoch {} at {}'.format(epoch+1, checkpoint_path))
    weight_utils.save_weights_as_pickle(checkpoint_path, optimizer, transformer)
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

No available checkpoints.
Epoch 1 Batch 0 Loss 1.8614 Accuracy 0.0000
Epoch 1 Batch 50 Loss 1.7295 Accuracy 0.0000
Epoch 1 Batch 100 Loss 1.7155 Accuracy 0.0024
Epoch 1 Batch 150 Loss 1.6967 Accuracy 0.0033
Epoch 1 Batch 200 Loss 1.6689 Accuracy 0.0045
Epoch 1 Batch 250 Loss 1.6408 Accuracy 0.0077
Epoch 1 Batch 300 Loss 1.6067 Accuracy 0.0112
Epoch 1 Batch 350 Loss 1.5681 Accuracy 0.0143
Epoch 1 Batch 400 Loss 1.5273 Accuracy 0.0169
Epoch 1 Batch 450 Loss 1.4836 Accuracy 0.0193
Epoch 1 Batch 500 Loss 1.4424 Accuracy 0.0212
Epoch 1 Batch 550 Loss 1.4037 Accuracy 0.0229
Epoch 1 Batch 600 Loss 1.3681 Accuracy 0.0242
Epoch 1 Batch 650 Loss 1.3340 Accuracy 0.0257
Epoch 1 Batch 700 Loss 1.3019 Accuracy 0.0273
Epoch 1 Batch 750 Loss 1.2702 Accuracy 0.0290
Epoch 1 Loss 1.2437 Accuracy 0.0305
Time taken for 1 epoch: 91.80832076072693 secs

Epoch 2 Batch 800 Loss 0.7600 Accuracy 0.0576
Epoch 2 Batch 850 Loss 0.7611 Accuracy 0.0581
Epoch 2 Batch 900 Loss 0.7525 Accuracy 0.0587
Epoch 2 Batch 950 L